In [1]:
import torch
import sys

def check_env_simple():
    # 提取 PyTorch 关联的 CUDA 版本
    cuda_ver = torch.version.cuda
    # 获取 Python 主版本号
    py_ver = f"{sys.version_info.major}.{sys.version_info.minor}"
    # 获取 PyTorch 版本（去除后缀）
    torch_ver = torch.__version__.split('+')[0]
    # 获取 ABI 状态
    abi_status = torch._C._GLIBCXX_USE_CXX11_ABI

    print("--- 当前环境核心参数 ---")
    print(f"Python 版本:  {py_ver}")
    print(f"PyTorch 版本: {torch_ver}")
    print(f"CUDA 版本:    {cuda_ver}")
    print(f"CXX11 ABI:    {abi_status}")
    print("-----------------------")

if __name__ == "__main__":
    check_env_simple()

--- 当前环境核心参数 ---
Python 版本:  3.11
PyTorch 版本: 2.8.0
CUDA 版本:    12.6
CXX11 ABI:    True
-----------------------


In [3]:
!wget https://github.com/Dao-AILab/flash-attention/releases/download/v2.8.3/flash_attn-2.8.3+cu12torch2.8cxx11abiTRUE-cp311-cp311-linux_x86_64.whl

--2025-12-26 14:58:41--  https://github.com/Dao-AILab/flash-attention/releases/download/v2.8.3/flash_attn-2.8.3+cu12torch2.8cxx11abiTRUE-cp311-cp311-linux_x86_64.whl
Resolving github.com (github.com)... 100.65.138.175
Connecting to github.com (github.com)|100.65.138.175|:443... connected.
HTTP request sent, awaiting response... ^C


In [4]:
import torch
import flash_attn

# 1. 检查版本
print("✅ Flash Attention 版本:", flash_attn.__version__)

# 2. 核心功能检查：尝试调用算子（需在 GPU 环境下）
try:
    from flash_attn.flash_attn_interface import flash_attn_func
    
    # 创建模拟数据 (Batch size 1, Seq len 128, Heads 8, Dim 64)
    # Flash Attention 要求数据类型必须是 float16 或 bfloat16
    q = torch.randn(1, 128, 8, 64, device='cuda', dtype=torch.float16)
    k = torch.randn(1, 128, 8, 64, device='cuda', dtype=torch.float16)
    v = torch.randn(1, 128, 8, 64, device='cuda', dtype=torch.float16)
    
    # 运行算子
    out = flash_attn_func(q, k, v, dropout_p=0.0, softmax_scale=None, causal=True)
    
    print("🚀 算子运行测试: 成功！(Output shape: {})".format(out.shape))
except Exception as e:
    print("❌ 算子运行失败，报错信息:")
    print(e)

✅ Flash Attention 版本: 2.8.3
🚀 算子运行测试: 成功！(Output shape: torch.Size([1, 128, 8, 64]))


In [ ]:
import torch
from causal_conv1d import causal_conv1d_fn

# 准备测试数据 (Batch size, Channels, Length)
device = "cuda"
batch, channels, seqlen = 2, 16, 64
x = torch.randn(batch, channels, seqlen, device=device, dtype=torch.bfloat16)
weight = torch.randn(channels, 4, device=device, dtype=torch.bfloat16) # 假设 kernel_size=4
bias = torch.randn(channels, device=device, dtype=torch.bfloat16)

try:
    # 运行算子
    out = causal_conv1d_fn(x, weight, bias, activation="silu")
    print("✅ Causal-Conv1d 算子运行成功!")
    print(f"输出形状: {out.shape}") # 应为 [2, 16, 64]
except Exception as e:
    print(f"❌ 运行失败，错误原因: {e}")

✅ Causal-Conv1d 算子运行成功!
输出形状: torch.Size([2, 16, 64])


: 